# Extracting Data From the WMO app

## 1 Insturactions of the WMO

Use the following steps to carry out the project
- Use this link (https://community.wmo.int/en/members/profiles to extract data on
heatwaves warning system/services, see example below, if heatwave is in the list of
warnings provided, then indicate a yes in the provided excel sheet (for the corresponding
country in column B in the excel sheet called “Heatwaves Warning Systems (WMO data)”).


full screenmode : https://app.powerbi.com/view?r=eyJrIjoiZjNhNzIzM2YtMjRkYS00ZjJjLWEzZmMtNmQzMGQzMDdiODU3IiwidCI6ImVhYTZiZTU0LTQ2ODctNDBjNC05ODI3LWMwNDRiZDhlOGQzYyIsImMiOjl9


## 2  Required Libraries

In [406]:
#The webdriver module in Selenium allows for automated control of web browsers.


from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains




## 3 Opening up the web browser and printing a sample text

The function below opens up chrome browser and waits for 60 minutes to load the data 

input: The website link

Output : the driver and wabdriverwait instance

In [442]:

def openChrome(link):
    # Options customize Chrome WebDriver behavior
    options = Options()

    # Initialize the WebDriver (e.g., Chrome)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    # Navigate to the desired URL
    driver.get(link)

    # wait for page to load
    wait = WebDriverWait(driver, 60)

    return driver, wait

In [443]:
link = "https://app.powerbi.com/view?r=eyJrIjoiZjNhNzIzM2YtMjRkYS00ZjJjLWEzZmMtNmQzMGQzMDdiODU3IiwidCI6ImVhYTZiZTU0LTQ2ODctNDBjNC05ODI3LWMwNDRiZDhlOGQzYyIsImMiOjl9"
driver_instance, wait  = openChrome(link)

Using the driver instance that we created print an sample text

In [444]:
# Locate all div elements with class "textbox"
div_elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="textbox"]')))

# Iterate over the list of elements and print the text content of each
for div_element in div_elements:
    print(div_element.text)
    print("-----")  # Just a separator for clarity

# driver_instance.quit()

Member Profiles
-----
The information contained in these profiles have been provided by WMO Members. The information is based on a self-assessment exercise. The date displayed in each section corresponds with the date in which the information was provided to WMO. 

If you find any issue or inconsistency with the information displayed, please contact community@wmo.int.
-----
Permanent Representative:
-----
Hydrological Adviser:
-----
National Meteorological Service:
-----
Responsible Ministry:
-----


## 4. Clicking on the Early Warning Services 

In [445]:
#EC stands for "Expected Conditions." It is a set of predefined conditions that the Selenium WebDriver uses.
#presence_of_element_located is one of many expected conditions (EC). It checks if an element is present on the DOM of a page.
def wait_and_click(driver, wait, css_selector):
    element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)))
    driver.execute_script("arguments[0].click();", element)

In [446]:
css_selector = 'a[title=" Early Warning Services"]'
wait_and_click(driver_instance, wait, css_selector)

## 5 - Get Warnings List

In [447]:
def saveWarnings(driver, css_selector):
    #'.title' is the class name of the element that contains the warning text
    warnings= driver_instance.find_elements(By.CSS_SELECTOR, css_selector)
    return warnings

warnings = saveWarnings(driver_instance, '.title')


In [448]:

for warning in warnings:
    print(warning.text)

Dust storm/Sandstorm
Fog
Freezing rain
Frost
Hail
Heat wave
Icing
Lightning
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Wild land fire/Forest fire
Wind
Yes
Yes
Yes
2 jours
Yes
Yes
No
Yes
Yes


## 6 selecting the next country

In [414]:

dropdown = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, '.slicer-dropdown-menu'))
)
dropdown.click()

# Wait for the dropdown options to be visible
dropdown_options = wait.until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '[role="option"]'))  # Assuming each dropdown option has a role of "option"
)

country_names = []
for option in dropdown_options:
    country_name = option.text
    country_names.append(country_name)

    # Uncomment the following lines if you want to click on a specific country
   
    if country_name == "Angola":  
        option.click()
        break
    

print(country_names)

['', '', '', '', '', '', '', '', '', '', '', '', 'Ground movement', 'Riverine flood', 'Flash flood', 'General flood', '', '', '', '', 'Ground moveme…', 'Riverine flood', 'General flood', 'Flash flood', 'Heat wave', 'Forest fire', 'General storm', 'General landslide', 'Coastal flood', 'Cold wave', 'Drought', 'Locust', '', '', '', '', '', '', '', '', '', '', '', '', 'Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola']


## 6 Get all the country names 

In [415]:

def getCountryNames(driver, wait):
    # Open the dropdown menu
    dropdown = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.slicer-dropdown-menu'))
    )
    dropdown.click()

    loaded_countries = set()
    while True:
        # Re-fetch the dropdown options in every iteration to avoid StaleElementReferenceException
        current_countries_elements = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.slicerText'))
        )
        
        # Extract country names from the current elements
        current_countries = {element.text for element in current_countries_elements}

        # Check if no new countries are found
        if not current_countries - loaded_countries:
            break
        
        loaded_countries.update(current_countries)

        # Scroll to the last element to load more countries
        ActionChains(driver).move_to_element(current_countries_elements[-1]).perform()
    print(loaded_countries)
    # Close the dropdown menu by clicking the dropdown button again
    dropdown.click()
    return loaded_countries

In [416]:
loaded_countries = getCountryNames(driver_instance, wait)

{'Solomon Islands', 'Ireland', 'Romania', 'Guyana', 'Turkmenistan', 'British Caribbean Territories', 'Monaco', 'United States of America', 'Cyprus', 'Andorra', 'Italy', 'Lithuania', 'Guatemala', 'Thailand', 'Fiji', 'Venezuela, Bolivarian Republic of', 'Eswatini', "Lao People's Democratic Republic", 'Belarus', 'Guinea', 'Latvia', 'India', 'Portugal', 'Qatar', 'Republic of Korea', 'Bulgaria', 'North Macedonia', 'Slovakia', 'United Republic of Tanzania', 'Iceland', 'Montenegro', 'Uganda', 'Austria', 'Mali', 'Armenia', 'Cameroon', 'Greece', 'Costa Rica', 'Israel', 'Bahrain', 'Sweden', 'Liberia', 'Afghanistan', 'Myanmar', 'Nicaragua', 'Tuvalu', 'Saudi Arabia', 'Lebanon', 'Cambodia', 'Ghana', 'Botswana', 'British Caribbean Territories - Anguilla', 'Congo', 'Germany', 'Antigua and Barbuda', 'British Caribbean Territories - Montserrat', 'Brazil', 'New Zealand', 'Czechia', 'Cabo Verde', 'Micronesia (Federated States of)', 'Central African Republic', 'French Polynesia', 'Angola', 'Cuba', 'Hondur

In [423]:
print(len(loaded_countries ))

198


## 7 Selecting A country

In [449]:
def select_country_from_dropdown(driver, wait, target_country):
    # Open the dropdown menu
    dropdown = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.slicer-dropdown-menu'))
    )
    dropdown.click()

    # Clear the search input
    search_input = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.searchInput'))
    )
    search_input.clear()

    loaded_countries = set()
    country_selected = False

    while True:
        # Re-fetch the dropdown options in every iteration to avoid StaleElementReferenceException
        current_countries_elements = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.slicerText'))
        )
        
        # Extract country names from the current elements
        current_countries = {element.text for element in current_countries_elements}

        # Check if target country is in the current batch of loaded countries
        if target_country in current_countries:
            for option in current_countries_elements:
                if option.text == target_country:
                    option.click()
                    country_selected = True
                    break

        # If the country has been selected, break out of the loop
        if country_selected:
            break
        
        # Check if no new countries are found
        if not current_countries - loaded_countries:
            break
        
        loaded_countries.update(current_countries)

        # Scroll to the last element to load more countries
        ActionChains(driver).move_to_element(current_countries_elements[-1]).perform()

    return country_selected


In [425]:
success = select_country_from_dropdown(driver_instance, wait, "Australia")
if success:
    print("has been selected successfully!")
else:
    print("Failed to select!")

has been selected successfully!


## selecting 8 country and getting the information

1- Initiate global variables

    - initiate a dictinary

    - driver, wait

2- Load all the country names

3 -Click on Early warning systems

----- repeat this 8 times ----

4- Click on the desired country name

5- extract warning systems in a list

   (if the list is empty)
   
6- save country name and the list in the map

In [426]:
# Convert the set to a list
countries_list = list(loaded_countries)

# Sort the list
countries_list.sort()

print(countries_list)
test= countries_list[0:8]

['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia (Plurinational State of)', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'British Caribbean Territories', 'British Caribbean Territories - Anguilla', 'British Caribbean Territories - British Virgin Islands', 'British Caribbean Territories - Cayman Islands', 'British Caribbean Territories - Montserrat', 'British Caribbean Territories - Turks and Caicos Islands', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', 'Croatia', 'Cuba', 'Curaçao and Sint Maarten', 'Cyprus', 'Czechia', "Côte d'Ivoire", "Democratic People's Republic of Korea", 'Democratic Republic of the Cong

In [427]:
test_list = ['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia']
print(len(test_list))
my_dict = {}

8


In [430]:
for country in test_list:
    print(country)
    success = select_country_from_dropdown(driver_instance, wait, country)
    if success:
        print("country been selected successfully!")
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict[country] = [0] 
    else:
        print("Failed to select the country!")
        


Afghanistan
country been selected successfully!
No warnings found for this country.
Albania
country been selected successfully!
No warnings found for this country.
Algeria
country been selected successfully!
Dust storm/Sandstorm
Fog
Freezing rain
Frost
Hail
Heat wave
Icing
Lightning
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Wild land fire/Forest fire
Wind
Yes
Yes
Yes
2 jours
Yes
Yes
No
Yes
Yes
Dust storm/Sandstorm
Fog
Freezing rain
Frost
Hail
Heat wave
Icing
Lightning
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Wild land fire/Forest fire
Wind
Yes
Yes
Yes
2 jours
Yes
Yes
No
Yes
Yes
Andorra
country been selected successfully!
Dust storm/Sandstorm
Fog
Freezing rain
Frost
Hail
Heat wave
Icing
Lightning
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Wild land fire/Forest fire
Wind
Yes
Yes
Yes
2 jours
Yes
Yes
No
Yes
Yes
Dust storm/Sandstorm
Fog
Freezing rain
Frost
Hail
Heat wave
Icing
Lightning
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Wil

In [431]:
my_dict

{'Afghanistan': [0],
 'Albania': [0],
 'Algeria': ['Dust storm/Sandstorm',
  'Fog',
  'Freezing rain',
  'Frost',
  'Hail',
  'Heat wave',
  'Icing',
  'Lightning',
  'Rain/Wet Spell',
  'Snow',
  'Snowstorm',
  'Thunderstorms/Squall lines',
  'Wild land fire/Forest fire',
  'Wind',
  'Yes',
  'Yes',
  'Yes',
  '2 jours',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'Yes'],
 'Andorra': ['Dust storm/Sandstorm',
  'Fog',
  'Freezing rain',
  'Frost',
  'Hail',
  'Heat wave',
  'Icing',
  'Lightning',
  'Rain/Wet Spell',
  'Snow',
  'Snowstorm',
  'Thunderstorms/Squall lines',
  'Wild land fire/Forest fire',
  'Wind',
  'Yes',
  'Yes',
  'Yes',
  '2 jours',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'Yes'],
 'Angola': ['Drought/Dry spell',
  'Flood',
  'Fog',
  'Lightning',
  'Rain/Wet Spell',
  'Wind',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'Yes',
  'No',
  'No'],
 'Antigua and Barbuda': ['Drought/Dry spell',
  'Flood',
  'Fog',
  'Lightning',
  'Rain/Wet Spell',
  'Wind',
  'Yes',
  'Yes',
  'No',
  'Y

In [458]:
my_dict

{'Afghanistan': [0],
 'Albania': [0],
 'Algeria': ['Dust storm/Sandstorm',
  'Fog',
  'Freezing rain',
  'Frost',
  'Hail',
  'Heat wave',
  'Icing',
  'Lightning',
  'Rain/Wet Spell',
  'Snow',
  'Snowstorm',
  'Thunderstorms/Squall lines',
  'Wild land fire/Forest fire',
  'Wind',
  'Yes',
  'Yes',
  'Yes',
  '2 jours',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'Yes'],
 'Andorra': ['Dust storm/Sandstorm',
  'Fog',
  'Freezing rain',
  'Frost',
  'Hail',
  'Heat wave',
  'Icing',
  'Lightning',
  'Rain/Wet Spell',
  'Snow',
  'Snowstorm',
  'Thunderstorms/Squall lines',
  'Wild land fire/Forest fire',
  'Wind',
  'Yes',
  'Yes',
  'Yes',
  '2 jours',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'Yes'],
 'Angola': ['Drought/Dry spell',
  'Flood',
  'Fog',
  'Lightning',
  'Rain/Wet Spell',
  'Wind',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'Yes',
  'No',
  'No'],
 'Antigua and Barbuda': ['Drought/Dry spell',
  'Flood',
  'Fog',
  'Lightning',
  'Rain/Wet Spell',
  'Wind',
  'Yes',
  'Yes',
  'No',
  'Y

## 9 adjusting so we can run the full loop

In [469]:
import time

def select_country_from_dropdown(driver, wait, target_country):
    # Open the dropdown menu
    dropdown = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.slicer-dropdown-menu'))
    )
    dropdown.click()

    # Clear the search input
    search_input = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.searchInput'))
    )
    search_input.clear()

    loaded_countries = set()
    country_selected = False

    while True:
        # Re-fetch the dropdown options in every iteration to avoid StaleElementReferenceException
        current_countries_elements = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.slicerText'))
        )
        
        # Extract country names from the current elements
        current_countries = {element.text for element in current_countries_elements}

        # Check if target country is in the current batch of loaded countries
        if target_country in current_countries:
            for option in current_countries_elements:
                if option.text == target_country:
                    option.click()
                    country_selected = True
                    break

        # If the country has been selected, break out of the loop
        if country_selected:
            break
        
        # Check if no new countries are found
        if not current_countries - loaded_countries:
            break
        
        loaded_countries.update(current_countries)

        # Forcefully scroll the dropdown using JavaScript
        driver.execute_script('arguments[0].scrollIntoView(true);', current_countries_elements[-1])
        time.sleep(2)  # wait for 2 seconds for potential loading

    return country_selected


In [461]:
test_list2= ["Bahrain", "Bangladesh", "Barbados", "Belarus", "Belgium", "Belize", "Benin", "Bhutan"]

In [472]:
success = select_country_from_dropdown(driver_instance, wait, "Benin")
if success:
    print("has been selected successfully!")
else:
    print("Failed to select!")

has been selected successfully!


In [475]:
for country in test_list2:
    print(country)
    success = select_country_from_dropdown(driver_instance, wait, country)
    if success:
        print("country been selected successfully!")
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict[country] = [0] 
    else:
        print("Failed to select the country!")
        


Bahrain
country been selected successfully!
No warnings found for this country.
Bangladesh
country been selected successfully!
No warnings found for this country.
Barbados
country been selected successfully!
No warnings found for this country.
Belarus
country been selected successfully!
Drought/Dry spell
Flood
Haze/Smoke
High Seas/Rogue waves etc.
Lightning
Rain/Wet Spell
Storm surge/Coastal flood
Thunderstorms/Squall lines
Tornado
Tropical cyclone
Tsunami
Volcanic ash
Wind
Yes
Yes
Yes
48 hours
Yes
Yes
Yes
Yes
No
This would need to be assessed


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=117.0.5938.88); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x000000010692fe58 chromedriver + 5090904
1   chromedriver                        0x0000000106926bc3 chromedriver + 5053379
2   chromedriver                        0x00000001064c2527 chromedriver + 447783
3   chromedriver                        0x00000001064c899d chromedriver + 473501
4   chromedriver                        0x00000001064ca8d4 chromedriver + 481492
5   chromedriver                        0x00000001064caa1c chromedriver + 481820
6   chromedriver                        0x000000010650aea2 chromedriver + 745122
7   chromedriver                        0x0000000106538662 chromedriver + 931426
8   chromedriver                        0x0000000106504

In [ ]:
loaded_countries

In [ ]:
len(loaded_countries)

stop when you see Spain

In [ ]:
# Convert the set to a list
countries_list = list(loaded_countries)

# Sort the list
countries_list.sort()

print(countries_list)
test= countries_list[0:9]

In [ ]:
success = select_country_from_dropdown(driver, "Austria")
if success:
    print("has been selected successfully!")
else:
    print("Failed to select!")


In [ ]:
test= ['Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Bahrain']


In [ ]:
import time
count = 0

for country in test:
    print(country)
    success = select_country_from_dropdown(driver, country)
    if success:
        print("country been selected successfully!")
    else:
        print("Failed to select the country!")
    count += 1
        # Brief sleep
    time.sleep(5)
    print(count)
    

# success = select_country_from_dropdown(driver, "Iceland")
# if success:
#     print("country been selected successfully!")
# else:
#     print("Failed to select the country!")
